## Chapter 7: Functional Programming

This chapter covers the introduction to functional programming.  In short, functional programming languages have functions as a important part of the language and apply and compose functions to create programs.

### 7.1: Functional vs. Non-functional forms

Here is a simple example about create an array from another array in both non-functional and functional forms:

In [ ]:
v=collect(1:5)

This is the traditional (non-functional form) of making a new array of squares of the original.

In [ ]:
v2=zeros(Int,5) # this an array of zeros of length 5
for i=1:5
  v2[i]=v[i]^2
end
v2

Here are some functional form versions instead:

In [ ]:
f(x)=x^2
map(f,v)

This is the same with an anonymous function

In [ ]:
map(x->x^2,v)

And recall that we did this in the previous chapter with:

In [ ]:
v.^2

The `map` function starts with an array, applies a function to each element and returns the new array.  Array an input and array as output.


#### Exercise
create a vector from 1 to 10 and 1) make a new array of `[1, 0.5, 0.333, 0.25, ...]`  and 2) make another with `[1 1//2 1//3 1//4 ...`] using the `map` function and the anonymous function notation.  Can you also get the same result with broadcasting?

### 7.2: Reducing an array

Another very common idea with arrays is to start with an array and reduce it to a single number.  The classic example is a sum. 

In [3]:
A=[1,2,3,4,5]

5-element Vector{Int64}:
 1
 2
 3
 4
 5

To do this, we use the `reduce` function, which has the same form as the `map` function except the function has two arguments. 

In [4]:
reduce((x,y)-> x+y, A)

15

To see really what's going on, here's a debugging version of that:

In [5]:
function f(tot,val)
  @show tot,val
  tot+val
end

f (generic function with 1 method)

In [6]:
reduce(f,A)

(tot, val) = (1, 2)
(tot, val) = (3, 3)
(tot, val) = (6, 4)
(tot, val) = (10, 5)


15

Notice that the first variable keeps the running sum.  Also, notice that the first call, when `val=1` is missing.  This is because the first value needs the function applied to something (it's binary).  If you don't put in an initial value, (often) it is initialized to 0 and a sum is applied on the first step.  Here's more what's going on.

In [7]:
reduce(f,A,init=0)

(tot, val) = (0, 1)
(tot, val) = (1, 2)
(tot, val) = (3, 3)
(tot, val) = (6, 4)
(tot, val) = (10, 5)


15

In [ ]:
reduce((x,y)->x*y,A)

If we have an array that we want to count the number of position element in the array, then here's a nice way to do this with `reduce`.  Again, this array as an input, value as an output.

In [ ]:
numPos(arr::Array{Int64,1}) = reduce((num,val) -> val > 0 ? num+1 : num, arr, init=0)

In [ ]:
numPos([-3,5,8,-2,11])

#### Exercise

Use reduce to take an array of strings, say `["The","dog","bit","the","cat"]` and concatenate all of the strings. (Can you figure out how to add a space between words?)

#### Summary of reduce

If you have an 1D array (vector) of numbers/strings (or anything) and you want to summarize the vector with a single value (number, string), you can often use reduced to do this. 

### 7.3: The mapreduce function

A super-handy funtion is the mapreduce function.  It's a combination of mapping an array to another array and then reducing all in one.  Consider the following that takes an array, squares each element and then sums:

In [ ]:
mapreduce(x->x^2,+,[1,2,3])

Since the function is the sum, this is also built into the sum command:

In [ ]:
sum(x->x^2,[1,2,3])

This example, takes an array of strings, and find the average word length:

In [ ]:
mapreduce(str->length(str),+,["This","is","a","very","boring","sentence"])/6

### 7.4: Mapping a Function over an 2D array

If we have a 2D array, we can map over the rows or columns of the array using the `mapslices` function.  The result is a 1D array (vector).

In [ ]:
A=[i+j for i=1:10,j=1:3]

This is the column sums

In [ ]:
mapslices(sum,A,dims=1)

And here are the row sums:

In [ ]:
mapslices(sum,A,dims=2)

### Summary of Chapter 7: Functional Programming

* What is functional programming and how does it differ for non-functional programming
* What are anonymous functions and how are they used.
* Using the map function to get a new array from an old one.
* Using the reduce function to get a single value from an array.
* Using the map function on multiple arrays
* Using the `mapreduce` and `mapslices` 